# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,yellowknife,62.4560,-114.3525,20.31,45,40,4.63,CA,1691447753
1,1,timbuktu,16.7735,-3.0074,34.52,34,12,4.58,ML,1691447993
2,2,papatowai,-46.5619,169.4708,5.56,79,100,6.87,NZ,1691447938
3,3,saipan,15.1355,145.7010,28.14,100,75,0.45,MP,1691447937
4,4,adamstown,-25.0660,-130.1015,20.59,74,77,3.49,PN,1691447931


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points("Lng", 
                                      "Lat", 
                                      geo = True,
                                      color = "City",
                                      alpha = 0.8,
                                      frame_width = 500,
                                   frame_height = 400,
                                      size = "Humidity",
                                      tiles = "OSM"
                                     )
# Display the map
city_map


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
new_city_df = city_data_df.loc[(city_data_df["Max Temp"] < 27)
                                       & (city_data_df["Max Temp"] > 21) 
                                       & (city_data_df["Cloudiness"] < 3)
                                       & (city_data_df["Wind Speed"] < 4.5)
                                      ]
# Drop any rows with null values
new_city_df.dropna()

# Display sample data
new_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
39,39,los frentones,-26.4077,-61.4137,21.02,44,0,4.12,AR,1691448407
59,59,ribeira grande,38.5167,-28.7000,23.21,84,1,2.24,PT,1691447950
79,79,chibougamau,49.9168,-74.3659,22.79,49,0,2.57,CA,1691447985
115,115,maumere,-8.6199,122.2111,24.14,73,1,2.33,ID,1691448419
236,236,argostolion,38.1811,20.4890,22.64,69,0,3.44,GR,1691448441
237,237,ilhabela,-23.7781,-45.3581,24.98,50,2,1.50,BR,1691448441
239,239,peruibe,-24.3200,-46.9983,21.11,80,0,4.36,BR,1691448442
246,246,axim,4.8699,-2.2405,24.33,88,2,3.60,GH,1691448298
258,258,yafran,32.0633,12.5286,22.53,55,0,2.33,LY,1691448445
308,308,kigoma,-4.8769,29.6267,23.79,60,0,1.45,TZ,1691448454


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = new_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df


,City,Country,Lat,Lng,Humidity,Hotel Name
39,los frentones,AR,-26.4077,-61.4137,44,
59,ribeira grande,PT,38.5167,-28.7000,84,
79,chibougamau,CA,49.9168,-74.3659,49,
115,maumere,ID,-8.6199,122.2111,73,
236,argostolion,GR,38.1811,20.4890,69,
237,ilhabela,BR,-23.7781,-45.3581,50,
239,peruibe,BR,-24.3200,-46.9983,80,
246,axim,GH,4.8699,-2.2405,88,
258,yafran,LY,32.0633,12.5286,55,
308,kigoma,TZ,-4.8769,29.6267,60,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000 
params = {
  "categories": "accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] =  f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address =  name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
los frentones - nearest hotel: No hotel found
ribeira grande - nearest hotel: Hospedaria JSF
chibougamau - nearest hotel: No hotel found
maumere - nearest hotel: Bogor
argostolion - nearest hotel: Ainos Hotel
ilhabela - nearest hotel: Vila Kebaya
peruibe - nearest hotel: Brisamar
axim - nearest hotel: Axim Beach Hotel
yafran - nearest hotel: فندق يفرن السياحي
kigoma - nearest hotel: sunset vista hotel
capinopolis - nearest hotel: Hotel Di Roma
al jawf - nearest hotel: No hotel found
san sebastian de la gomera - nearest hotel: Hotel Villa Gomera
sarakhs - nearest hotel: هتل ابریشم
bayindir - nearest hotel: Su Otel
kos - nearest hotel: MARITINA
doganhisar - nearest hotel: No hotel found
staroshcherbinovskaya - nearest hotel: Гостиница "Белая Акация"
lata - nearest hotel: Residencial Botânico
tomar - nearest hotel: Hotel Kamanga


,City,Country,Lat,Lng,Humidity,Hotel Name
39,los frentones,AR,-26.4077,-61.4137,44,No hotel found
59,ribeira grande,PT,38.5167,-28.7000,84,Hospedaria JSF
79,chibougamau,CA,49.9168,-74.3659,49,No hotel found
115,maumere,ID,-8.6199,122.2111,73,Bogor
236,argostolion,GR,38.1811,20.4890,69,Ainos Hotel
237,ilhabela,BR,-23.7781,-45.3581,50,Vila Kebaya
239,peruibe,BR,-24.3200,-46.9983,80,Brisamar
246,axim,GH,4.8699,-2.2405,88,Axim Beach Hotel
258,yafran,LY,32.0633,12.5286,55,فندق يفرن السياحي
308,kigoma,TZ,-4.8769,29.6267,60,sunset vista hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   geo = True,
                                   color = "City",
                                   alpha = 0.8,
                                   frame_width = 500,
                                   frame_height = 400,
                                   size = "Humidity",
                                   tiles = "OSM",
                                   hover_cols = ["Hotel Name", "Country"]
                                  )

# Display the map
hotel_map


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)